In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from lxml import html
import requests

import warnings
warnings.filterwarnings("ignore")

In [9]:
def wrangle_hit(file, min):
    df = pd.read_csv(file)
    df = df[(df["PA"] >= min)] # Playing time threshold

    # Required change for Splits Leaderboard compatibility
    if "Tm" in df.columns:
        df = df.rename(columns = {"Tm": "Team", "playerId": "playerid"})
    df = df.set_index("playerid")
    
    # Create rate stats
    df["BB%"] = round(df["BB"]/df["PA"],3)
    df["K%"] = round(df["SO"]/df["PA"],3)
    df["SLG%"] = round((df["1B"]+df["2B"]*2+df["3B"]*3+df["HR"]*4)/df["AB"],3)
    df["BIP%"] = round((df["AB"]+df["SF"]-df["SO"])/df["PA"],3)
    df["HIT"] = round(df["SLG%"]*df["BIP%"],3)
    
    # Find league averages and standard deviations
    league = df.agg({"SLG%" : ["mean", "std"],
                     "BIP%" : ["mean", "std"],
                     "HIT" : ["mean", "std"],
                     "BB%" : ["mean", "std"],
                     "K%" : ["mean", "std"]})
    
    # Get Z-scores for rate stats
    df["zBB%"] = round((df["BB%"]-league["BB%"]["mean"])/league["BB%"]["std"],2)
    df["zK%"] = round(-(df["K%"]-league["K%"]["mean"])/league["K%"]["std"],2)
    df["zSLG%"] = round((df["SLG%"]-league["SLG%"]["mean"])/league["SLG%"]["std"],2)
    df["zBIP%"] = round((df["BIP%"]-league["BIP%"]["mean"])/league["BIP%"]["std"],2)
    df["Z"] = round((df["HIT"]-league["HIT"]["mean"])/league["HIT"]["std"],2)
    df["wZ"] = df["Z"]*df["PA"]
    
    # Add Weighted Z to aggregate variable
    league["wZ"] = df.agg({"wZ" : ["mean", "std"]})
    
    # Get Z-score for Weighted Z
    df["wZ"] = round((df["wZ"]-league["wZ"]["mean"])/league["wZ"]["std"],2)
    

    
    return df[["Name", "Team", "PA", "BB%", "K%", "SLG%", "BIP%", "zBB%",
               "zK%", "zSLG%", "zBIP%", "Z", "wZ"]].sort_values(by="wZ", ascending=False)

hitters = wrangle_hit("../../../../Downloads/Fangraphs Leaderboard.csv", 70)
len(hitters)

415

In [10]:
def missing_team(df):
    '''
    Checks that there are 30 teams in a dataframe;
    Returns players missing team values.
    '''
    if len(df["Team"].unique()) != 30:
        return df[(df["Team"] == "- - -")]
    else:
        print("All hitters have a home!")
        
# Fill in missing team data
hitters["Team"][1177]  = "LAD" # Albert Pujols
hitters["Team"][15654] = "NYM" # Billy McKinney
hitters["Team"][15986] = "MIL" # Willy Adames
hitters["Team"][16448] = "SEA" # Jacob Nottingham
hitters["Team"][12160] = "PIT" # Ben Gamel
hitters["Team"][13324] = "ARI" # Ildemaro Vargas
hitters["Team"][15194] = "SEA" # Jake Bauers
hitters["Team"][18136] = "PIT" # Ka'ai Tom
hitters["Team"][15274] = "SFG" # Mike Tauchman
hitters["Team"][27459] = "LAD" # Yoshi Tsutsugo
hitters["Team"][12284] = "ATL" # Kevan Smith
hitters["Team"][10762] = "TOR" # Corey Dickerson
hitters["Team"][11936] = "MIA" # Joe Panik
hitters["Team"][11899] = "ATL" # Joc Pederson
hitters["Team"][16852] = "PIT" # John Nogowski
hitters["Team"][5000]  = "ATL" # Stephen Vogt
hitters["Team"][13185] = "ATL" # Orlando Arcia
hitters["Team"][15402] = "LAA" # Jack Mayfield
hitters["Team"][3648]  = "WSN" # Rene Rivera
hitters["Team"][15679] = "MIL" # Rowdy Telez
hitters["Team"][17468] = "BAL" # Kelvin Gutierrez
hitters["Team"][16404] = "BAL" # Domingo Leyba
hitters["Team"][11205] = "LAA" # Adam Eaton
hitters["Team"][15124] = "NYY" # Tim Locastro

    
missing_team(hitters)

All hitters have a home!


In [11]:
# Create splits dataframes
rhh_rhp = wrangle_hit("../../../../Downloads/Splits Leaderboard Data as RHH vs RHP.csv", 20)
rhh_lhp = wrangle_hit("../../../../Downloads/Splits Leaderboard Data as RHH vs LHP.csv", 20)
lhh_rhp = wrangle_hit("../../../../Downloads/Splits Leaderboard Data as LHH vs RHP.csv", 20)
lhh_lhp = wrangle_hit("../../../../Downloads/Splits Leaderboard Data as LHH vs LHP.csv", 10)

In [13]:
# Add preferred batting side to main dataframe
Bat = []
for i in hitters.index:
    if i in rhh_lhp.index and i in lhh_rhp.index:
        Bat.append("S")
    elif i in rhh_lhp.index or i in rhh_rhp.index:
        Bat.append("R")
    elif i in lhh_lhp.index or i in lhh_rhp.index:
        Bat.append("L")
    else:
        Bat.append("?")
        
hitters["Bat"] = Bat
# hitters.head(20)

In [39]:
# Add matchup strengths vs righties and lefties
vrhp = pd.concat([rhh_rhp, lhh_rhp])
vrhp = vrhp.rename(columns={"wZ": "vR"})
vrhp = vrhp[(vrhp.index.isin(hitters.index) == 1)]["vR"]

vlhp = pd.concat([rhh_lhp, lhh_lhp])
vlhp = vlhp.rename(columns={"wZ": "vL"})
vlhp = vlhp[(vlhp.index.isin(hitters.index) == 1)]["vL"]

if "vR" not in hitters.columns:
    hitters = pd.merge(hitters, vrhp, how="outer", left_index=True, right_index=True)
if "vL" not in hitters.columns:
    hitters = pd.merge(hitters, vlhp, how="outer", left_index=True, right_index=True).sort_values(by="wZ", ascending=False)
    
# hitters.isna().sum()

In [52]:
def get_injury_report(site):
    """
    Retrieves MLB injury information from the CBS Sports website.
    """
    
    page = requests.get(site)
    tree = html.fromstring(page.content)
    
    # Clean and organize the data
    players, positions, injuries, return_dates = [], [], [], []
    for i in range(1,20):
        player = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[1]/span[2]/span/a/text()")
        [players.append(j) for j in player]

        position = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[2]/text()")
        for j in position:
            positions.append(j.split()[0])

        injury = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[4]/text()")
        for j in injury:
            injuries.append(j.split()[0])

        return_date = tree.xpath(f"//*[@id='TableBase']/div/div/table/tbody/tr[{i}]/td[5]/text()")
        for j in return_date:
            if j.split()[-1] == "season":
                return_dates.append(datetime.strptime("Nov 1 2021", "%b %d %Y"))
            else:
                return_dates.append(datetime.strptime(j.split()[-2]+" "+j.split()[-1]+" 2021","%b %d %Y"))
    
    # Create a dataframe with the organized data
    df = pd.DataFrame({"Name": players, "Pos": positions, "Return": return_dates,
                       "Injury": injuries})

    length = df["Return"] - datetime.today()
    length = [i.round("D") for i in length]
    df["Length"] = length
    df = df[(df["Length"] > "0 days")]
    
    return df

injury_report = get_injury_report("https://www.cbssports.com/mlb/injuries/")
# injury_report[~(injury_report["Pos"].str.endswith("P"))]

In [41]:
def remove_inj_players(df):
    """
    Removes injured players from a dataframe.
    """
    df["Inj"] = df.Name.isin(injury_report.Name)
    df = df[(df["Inj"] == 0)]
    df = df.drop("Inj", axis=1)
    return df
filtered_hitters = remove_inj_players(hitters)
# filtered_hitters

In [42]:
tm_players = {}
player_val = {}
sort_value = "wZ"
for tm in sorted(filtered_hitters["Team"].unique()):
    tm_players[tm] = []
    player_val[tm] = []
    for j, i, v in zip([x+1 for x in range(len(filtered_hitters))], filtered_hitters.index, filtered_hitters["Team"]):
        if tm == v:
            tm_players[tm].append(j)
            player_val[tm].append(filtered_hitters[sort_value][i])
            
# tm_players

In [44]:
tm_player_rks = pd.DataFrame(tm_players.values(), tm_players.keys(),
                             [j+1 for j in range(max([len(tm_players[i]) for i in tm_players.keys()]))]
                            ).fillna(0).astype(int)

# tm_player_rks.sort_values(by=3)

In [45]:
player_val_rks = round(pd.DataFrame(player_val.values(), player_val.keys(),
                             [j+1 for j in range(max([len(tm_players[i]) for i in tm_players.keys()]))]
                            ).fillna(0).astype(float).cumsum(axis=1),2)

player_val_rks.sort_values(by=7, ascending=False)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
TOR,4.56,6.72,8.53,10.29,11.40,12.43,12.81,12.62,12.39,11.68,10.67,9.65,9.65,9.65,9.65,9.65
HOU,2.46,4.80,6.89,8.81,10.28,11.24,11.41,11.26,10.47,9.50,8.44,6.70,6.70,6.70,6.70,6.70
BOS,2.85,5.17,7.47,8.53,9.40,9.69,9.40,8.75,7.93,7.01,7.01,7.01,7.01,7.01,7.01,7.01
LAD,1.90,3.74,5.53,6.83,7.92,8.48,8.49,8.37,8.15,7.22,6.02,4.61,4.61,4.61,4.61,4.61
SDP,2.52,4.55,6.36,6.96,7.47,7.49,7.45,6.97,6.41,5.38,4.28,4.28,4.28,4.28,4.28,4.28
ATL,2.23,4.21,6.01,6.58,7.06,7.23,7.32,7.14,6.83,6.41,5.94,5.39,4.82,3.99,3.99,3.99
LAA,3.00,4.80,6.15,7.17,7.27,7.36,7.32,7.25,7.11,6.95,6.30,5.48,4.55,4.55,4.55,4.55
STL,2.73,3.81,4.84,5.50,6.01,6.27,6.02,5.52,4.78,3.84,2.73,1.42,1.42,1.42,1.42,1.42
MIN,2.20,3.34,4.17,4.68,5.11,5.51,5.73,5.51,4.75,3.96,3.16,2.30,1.42,0.46,0.46,0.46
SFG,1.91,3.26,4.12,4.75,5.10,5.39,5.33,5.22,5.08,4.92,4.47,3.75,2.40,2.40,2.40,2.40


In [53]:
team_lookup = "ATL"
filtered_hitters[(filtered_hitters["Team"] == team_lookup)].sort_values(by="wZ", ascending=False)

,Name,Team,PA,BB%,K%,SLG%,BIP%,zBB%,zK%,zSLG%,zBIP%,Z,wZ,Bat,vR,vL
playerid,,,,,,,,,,,,,,,,
16556,Ozzie Albies,ATL,385,0.078,0.182,0.496,0.732,-0.33,0.85,1.11,0.94,1.51,2.23,S,1.46,2.82
18401,Ronald Acuna Jr.,ATL,360,0.136,0.236,0.596,0.603,1.26,0.02,2.26,-0.69,1.45,1.98,R,2.31,1.07
5361,Freddie Freeman,ATL,402,0.142,0.179,0.507,0.674,1.43,0.90,1.24,0.20,1.19,1.80,L,2.66,0.92
18360,Austin Riley,ATL,368,0.101,0.253,0.473,0.628,0.30,-0.25,0.85,-0.38,0.51,0.57,R,1.38,-0.40
18314,Dansby Swanson,ATL,377,0.069,0.273,0.451,0.647,-0.58,-0.56,0.59,-0.14,0.44,0.48,R,0.83,0.33
11899,Joc Pederson,ATL,298,0.077,0.248,0.432,0.658,-0.36,-0.17,0.38,0.00,0.31,0.17,L,0.52,0.52
8418,Ehire Adrianza,ATL,148,0.108,0.182,0.425,0.696,0.49,0.85,0.30,0.48,0.50,0.09,S,0.30,-0.48
18721,Guillermo Heredia,ATL,225,0.089,0.249,0.419,0.636,-0.03,-0.18,0.23,-0.28,0.04,-0.18,R,-0.26,0.40
5000,Stephen Vogt,ATL,155,0.116,0.239,0.397,0.639,0.71,-0.03,-0.03,-0.24,-0.14,-0.31,L,-0.27,0.07


In [56]:
### TODO: Get playerid from href in html code for smoother
###       transition importing stats to lineups.

def get_lineups(site):
    """
    Creates a nested dictionary containing each
    team's starting pitcher and starting lineup.
    """
    # Retrieve information from site
    page = requests.get(site)
    tree = html.fromstring(page.content)
    
    # team, pitcher, & hitter variables provide html code values from the site
    lineups = {}
    for MU in range(1,20): # matchup variable
        for h_a in range(1,3): # home/away variable
            team = tree.xpath(f"/html/body/div[1]/div/main/div[2]/div[{MU}]/div[2]/div[1]/div/div[{h_a}]/div/text()")
            pitcher = tree.xpath(f"/html/body/div[1]/div/main/div[2]/div[{MU}]/div[2]/div[2]/ul[{h_a}]/li[1]/div[1]/a/text()")

            if team != []:
                lineups[team[0]] = {"SP": "", "lineup": []}
                
            if pitcher != []:
                lineups[team[0]]["SP"] += pitcher[0]
                
            for order in range(3,20): # batting order variable
                hitter = tree.xpath(f"/html/body/div[1]/div/main/div[2]/div[{MU}]/div[2]/div[2]/ul[{h_a}]/li[{order}]/a")
                
                if (hitter != []) and "(order%2 == 1)": # values start at 3rd html list and move up by two..sometimes
                    lineups[team[0]]["lineup"].append(hitter[0].get("title"))
        
    return lineups


lineups = get_lineups("https://www.rotowire.com/baseball/daily-lineups.php")
team = "BAL"
players = []
for tm in lineups:
    for player in lineups[tm]["lineup"]:
        if player != lineups[tm]["SP"]:
            players.append(player)
            
players = pd.Series(players)
missing_players = players[(players.isin(hitters["Name"]) == 0)]
lineups[team]["lineup"]

['Cedric Mullins',
 'Austin Hays',
 'Trey Mancini',
 'Ryan Mountcastle',
 'Anthony Santander',
 'Ramon Urias',
 'Pedro Severino',
 'Pat Valaika',
 'Kelvin Gutierrez']

In [16]:
hitters["Name"][14773] = "Yu Chang"
hitters["Name"][16885] = "Joshua Fuentes"
hitters["Name"][12984] = "Jackie Bradley"
hitters["Name"][18401] = "Ronald Acuna"
hitters["Name"][19709] = "Fernando Tatis"
hitters["Name"][19611] = "Vladimir Guerrero"
hitters["Name"][17929] = "Cedric Mullins"
hitters["Name"][19238] = "Lourdes Gurriel"
hitters["Name"][18126] = "LaMonte Wade"
hitters["Name"][11489] = "Michael Taylor"
hitters["Name"][16542] = "Shed Long"


In [18]:
hitters[(hitters["Name"] == "LaMonte Wade")]

,Name,Team,PA,BB%,K%,SLG%,BIP%,zBB%,zK%,zSLG%,zBIP%,Z,wZ,Bat,vR,vL,Inj
playerid,,,,,,,,,,,,,,,,,
18126,LaMonte Wade,SFG,110,0.082,0.182,0.505,0.7,-0.19,0.85,1.21,0.55,1.36,0.52,L,1.27,-0.77,False
